In [3]:
import base64
from anthropic import Anthropic
from IPython.display import Image

client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [30]:
user_query = str(input('Enter your question or query: '))

In [31]:
def retrieve_values_for_tools(query):
    return f"""You are given a sentence that contains a property, and you should generate the values in the form of a list of 3 strings, where 

    - the first string is the property name, 
    - the second string is the type of the property (str, int) and 
    - the third string is a short 1-line description of what the property is.

    Here are a couple of examples that illustrate this:

    <example>
    Sentence: "I enjoy reading sci-fi genre books".
    The output: ['book_genre_interest','str','The genre of books the user enjoys.']
    </example>

    <example>
    Sentence: "My goal today is to ensure that I can get my bicep-workout done.
    The output: ['today_goal','str',"The user's goal for today"]
    </example>

    Now perform the same action here:

    Sentence: {query}
    Output:
    """


In [33]:
retrieve_values_for_tools(user_query)

'You are given a sentence that contains a property, and you should generate the values in the form of a list of 3 strings, where \n\n    - the first string is the property name, \n    - the second string is the type of the property (str, int) and \n    - the third string is a short 1-line description of what the property is.\n\n    Here are a couple of examples that illustrate this:\n\n    <example>\n    Sentence: "I enjoy reading sci-fi genre books".\n    The output: [\'book_genre_interest\',\'str\',\'The genre of books the user enjoys.\']\n    </example>\n\n    <example>\n    Sentence: "My goal today is to ensure that I can get my bicep-workout done.\n    The output: [\'today_goal\',\'str\',"The user\'s goal for today"]\n    </example>\n\n    Now perform the same action here:\n\n    Sentence: I am really looking forward to watching Oppenheimer because I enjoy Nolan\'s movies.\n    Output:\n    '

Retrieving the list of values in order to feed into the 

In [34]:
response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=[{'role':'user','content':retrieve_values_for_tools(user_query)}],
)
print(response.content[0].text)

['anticipated_movie', 'str', "The movie the user is looking forward to watching."]


In [ ]:
def claude_response(query):
    response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=[{'role':'user','content':retrieve_values_for_tools(query)}],
    )
    return response.content[0].text

In [37]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 6.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.5/528.5 kB 8.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 12.2 MB/s eta 0:00:000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 14.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 8.1 M

In [38]:
from memory.associative_memory import AssociativeMemory
from memory.longterm_memory import LongtermSummaryMemory
from memory.procedural_memory import ProceduralMemory
from memory.working_memory import WorkingMemory
from llm import LLMClient
from memory.base_memory import BaseMemory

import chromadb
import uuid

In [45]:
import base64
from anthropic import Anthropic
from memory_manager import Memoria

In [49]:
user_query = str(input('Enter your query:'))

In [50]:
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"


def retrieve_values_for_tools(user_query):
    return f"""You are given a sentence that contains a property, and you should generate the values in the form of a list of 3 strings, where 

    - the first string is the property name, 
    - the second string is the type of the property (str, int) and 
    - the third string is a short 1-line description of what the property is.

    Here are a couple of examples that illustrate this:

    <example>
    Sentence: "I enjoy reading sci-fi genre books".
    The output: ['book_genre_interest','str','The genre of books the user enjoys.']
    </example>

    <example>
    Sentence: "My goal today is to ensure that I can get my bicep-workout done.
    The output: ['today_goal','str',"The user's goal for today"]
    </example>

    Now perform the same action here:

    Sentence: {user_query}
    Output:
    """


system_prompt = """
As an AI assistant, your task is to interact with the user and manage your memory effectively. To accomplish this, you have access to the MemoryManager class, which allows you to store and retrieve information in different types of memory: working memory, associative memory, long-term summary memory, and procedural memory.

Based on the user's input, you should determine which memory type to use, and whether to store or retrieve information. Here are the broad guidelines to follow:

1. If the user mentions their interests, hobbies, or preferences to you, store this information in associative memory using:
   memory_manager.store('associative', 'user_interests', 'summary_of_interests')

2. If the user's query requires knowledge about their interests, hobbies, or preferences, perform a retrieval into the associative memory using:
   memory_manager.retrieve('associative','user_query')
   
3. If the user provides a specific goal or task they want to accomplish, or information relevant to achieve the working goal, store it in working memory using:
   memory_manager.store('working', 'name_of_property', 'summary_of_information')

4. If the user's query requires context that references previous conversations, then query the long term memory to identify if there's relevant information available using:
    memory_manager.retrieve('longterm','summary_of_question')   

5. If the user shares a long-term plan or their objectives, store it in long-term summary memory using:
   memory_manager.store('longterm', 'name_of_property', 'summary_of_user_message')

6. If the user's query requires context that references previous conversations, then query the long term memory to identify if there's relevant information available using:
    memory_manager.retrieve('longterm','summary_of_question')

7. If the user provides step-by-step instructions or a procedure to follow, store it in procedural memory using:
   memory_manager.store('procedural', 'name_of_property', 'description_of_property')

8. If the user's goal requires you to take actions, then retrieve the tools you need to use through:
    memory_manager.retrieve('procedural','name_of_action')

Example 1: If the user provides you with the set of instructions to set up a Github repo, then you store it into procedural memory using
    memory_manager.store('procedural', 'Hackathon', "Setup the Git hub repo")
    
Example 2: If the user wants to retrieve the information related to hackathon setup, you retrieve from procedural memory to identify if there's relevant information using:
    memory_manager.retrieve('procedural', 'Hackathon')

Remember to use the information stored in memory to provide context-aware responses and assist the user effectively. If the user's input doesn't fit into any specific memory category, use your best judgment to determine if and where to store or retrieve the information.

Analyze their input and manage your memory accordingly to provide the best possible assistance.
"""


def claude_response_values(query):
    response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=[{'role':'user','content':retrieve_values_for_tools(query)}],
    )
    return response.content[0].text

claude_responses = claude_response_values(user_query)

def tools_for_claude(responses_from_claude):

    return [
        {
            "name": 'memory_manager.store',
            'description': 'Store relevant context into memory',
            'input_schema': {
                "type": 'object',
                'properties': {
                    'memory_type': {
                        'type': 'string',
                        'enum': ['procedural','longterm','working','associative'],
                        'description': 'The type of memory we want to store into'
                    },
                    responses_from_claude[0]: {
                        'type': responses_from_claude[1],
                        'description':responses_from_claude[2],
                    },
                    'user_query': {
                        'type':'str',
                        'description': 'Query of the user'
                    }
                    }
                },
                'required':['memory_type',responses_from_claude[0],'user_query'],
            },

        {
            "name": 'memory_manager.retrieve',
            'description': 'Retrieve relevant context into memory',
            'input_schema': {
                "type": 'object',
                'properties': {
                    'memory_type': {
                        'type': 'string',
                        'enum': ['procedural','longterm','working','associative'],
                        'description': 'The type of memory we want to retrieve from'
                    },
                    responses_from_claude[0]: {
                        'type': responses_from_claude[1],
                        'description':responses_from_claude[2],
                    },
                    }
                },
                'required':['memory_type',responses_from_claude[0]],
            },
    ]


In [54]:
import ast
claude_responses = ast.literal_eval(claude_responses)

In [55]:
claude_responses

['workout_enjoyment', 'str', 'The user enjoys working out and exercising.']

In [60]:
import anthropic

In [61]:
client = anthropic.Anthropic()

In [65]:
claude_responses

['workout_enjoyment', 'str', 'The user enjoys working out and exercising.']

In [62]:
def claude_action():
    response = client.beta.tools.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    tools = tools,
    system = system_prompt,
    messages=[{'role':'user','content':user_query}],
    )
    return response

In [63]:
claude_action()

AttributeError: 'Anthropic' object has no attribute 'beta'

In [64]:
tools

[{'name': 'memory_manager.store',
  'description': 'Store relevant context into memory',
  'input_schema': {'type': 'object',
   'properties': {'memory_type': {'type': 'string',
     'enum': ['procedural', 'longterm', 'working', 'associative'],
     'description': 'The type of memory we want to store into'},
    '[': {'type': "'", 'description': 'w'},
    'user_query': {'type': 'str', 'description': 'Query of the user'}}},
  'required': ['memory_type', '[', 'user_query']},
 {'name': 'memory_manager.retrieve',
  'description': 'Retrieve relevant context into memory',
  'input_schema': {'type': 'object',
   'properties': {'memory_type': {'type': 'string',
     'enum': ['procedural', 'longterm', 'working', 'associative'],
     'description': 'The type of memory we want to retrieve from'},
    '[': {'type': "'", 'description': 'w'}}},
  'required': ['memory_type', '[']}]